In [ ]:
!pip install mecab-python3

In [ ]:
!apt-get update

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [ ]:
!apt-get install g++ openjdk-8-jdk python-dev python3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
python3-dev is already the newest version (3.6.7-1~18.04).
openjdk-8-jdk is already the newest version (8u265-b01-0ubuntu2~18.04).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
!pip3 install JPype1-py3

In [ ]:
!pip3 install konlpy

In [ ]:
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
# 이건 우선 패스할 것!
# 만약에 밑에까지 다 했는데, 오류 뜨면 이거 돌려보고 다시 해볼 것!
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ 

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Mecab
import numpy as np
import re
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
cafe_data = pd.read_excel("/content/drive/My Drive/Colab Notebooks/데이터/F 카페(7,859).xlsx")
cafe_data.head()

,SPEAKER,SENTENCE,DATAID,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,아이스아메리카노 하나요,16334604,F,카페,카페,1,1,일반주문,NaN,Q,NaN,아이스 아메리카노 하나요,NaN,NaN,NaN,"아이스 아메리카노, 하나",NaN,NaN
1,점원,테이크아웃하실 건가요?,16334604,F,카페,카페,0,2,일반주문,테이크아웃여부,Q,NaN,NaN,테이크아웃하실 건가요?,NaN,NaN,테이크아웃,NaN,NaN
2,고객,네 텀블러에 넣어주세요,16334604,F,카페,카페,1,3,텀블러사용,NaN,Q,NaN,네 텀블러에 넣어주세요,NaN,NaN,NaN,텀블러,NaN,NaN
3,점원,텀블러 할인 300원 해드릴게요,16334604,F,카페,카페,0,4,텀블러사용,NaN,A,NaN,NaN,NaN,NaN,텀블러 할인 300원 해드릴게요,"텀블러, 할인, 300원",NaN,NaN
4,고객,그란데 사이즈로 주세요,16334604,F,카페,카페,1,5,일반주문,NaN,Q,NaN,그란데 사이즈로 주세요,NaN,NaN,NaN,그란데 사이즈,NaN,NaN


In [ ]:
# 단어 아니면 삭제
cafe_data['SENTENCE'] = cafe_data['SENTENCE'].str.replace("[^\w]|br", " ")
# cafe_data['text'] = cafe_data['text'].str.replace("[^\w]|br", " ")
cafe_data.head()
# cafe_data.tail()

,SPEAKER,SENTENCE,DATAID,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,아이스아메리카노 하나요,16334604,F,카페,카페,1,1,일반주문,NaN,Q,NaN,아이스 아메리카노 하나요,NaN,NaN,NaN,"아이스 아메리카노, 하나",NaN,NaN
1,점원,테이크아웃하실 건가요,16334604,F,카페,카페,0,2,일반주문,테이크아웃여부,Q,NaN,NaN,테이크아웃하실 건가요?,NaN,NaN,테이크아웃,NaN,NaN
2,고객,네 텀블러에 넣어주세요,16334604,F,카페,카페,1,3,텀블러사용,NaN,Q,NaN,네 텀블러에 넣어주세요,NaN,NaN,NaN,텀블러,NaN,NaN
3,점원,텀블러 할인 300원 해드릴게요,16334604,F,카페,카페,0,4,텀블러사용,NaN,A,NaN,NaN,NaN,NaN,텀블러 할인 300원 해드릴게요,"텀블러, 할인, 300원",NaN,NaN
4,고객,그란데 사이즈로 주세요,16334604,F,카페,카페,1,5,일반주문,NaN,Q,NaN,그란데 사이즈로 주세요,NaN,NaN,NaN,그란데 사이즈,NaN,NaN


In [ ]:
pre_cafe_data = cafe_data.loc[0:6673,["SPEAKER","SENTENCE"]]

In [ ]:
pre_cafe_data.head()

,SPEAKER,SENTENCE
0,고객,아이스아메리카노 하나요
1,점원,테이크아웃하실 건가요
2,고객,네 텀블러에 넣어주세요
3,점원,텀블러 할인 300원 해드릴게요
4,고객,그란데 사이즈로 주세요


In [ ]:
len(pre_cafe_data[pre_cafe_data["SPEAKER"]=="고객"]), len(pre_cafe_data[pre_cafe_data["SPEAKER"]=="점원"])

(3337, 3337)

In [ ]:
# 점원과 고객이 한말 두개 따로 나누기.
# Q and A 형식으로 dataframe 만들기.

user = pre_cafe_data[pre_cafe_data["SPEAKER"] == "고객"]["SENTENCE"]
service = pre_cafe_data[pre_cafe_data["SPEAKER"] == "점원"]["SENTENCE"]
display(user)
display(service)
print(type(user))

0           아이스아메리카노 하나요
2           네 텀블러에 넣어주세요
4           그란데 사이즈로 주세요
6         네 보고쿠폰으로 결제할게요
8                  괜찮습니다
              ...       
6664        비엔나는 어떤 거에요 
6666    카페라떼 아이스로 하나 주세요
6668     네 얼음은 조금만 넣어주세요
6670         원래하던데로 해주세요
6672        주차장은 앞에 있나요 
Name: SENTENCE, Length: 3337, dtype: object

1            테이크아웃하실 건가요 
3       텀블러 할인 300원 해드릴게요
5              적립카드 있으세요 
7            현금영수증 해드릴까요 
9         저쪽에서 잠시만 대기해주세요
              ...        
6665     아메리카노에 휘핑 들어갔습니다
6667         가지고 가시는 거에요 
6669         샷 더 넣어 드릴까요 
6671    연하다 싶으면 더 넣어 드릴게요
6673        주차장은 뒤에도 있습니다
Name: SENTENCE, Length: 3337, dtype: object

<class 'pandas.core.series.Series'>


In [ ]:
user.name="user"
service.name="service"

In [ ]:
service = pd.DataFrame(service).reset_index(drop=True)
user = pd.DataFrame(user).reset_index(drop=True)
result_data = pd.concat([user, service], axis=1)
result_data

,user,service
0,아이스아메리카노 하나요,테이크아웃하실 건가요
1,네 텀블러에 넣어주세요,텀블러 할인 300원 해드릴게요
2,그란데 사이즈로 주세요,적립카드 있으세요
3,네 보고쿠폰으로 결제할게요,현금영수증 해드릴까요
4,괜찮습니다,저쪽에서 잠시만 대기해주세요
...,...,...
3332,비엔나는 어떤 거에요,아메리카노에 휘핑 들어갔습니다
3333,카페라떼 아이스로 하나 주세요,가지고 가시는 거에요
3334,네 얼음은 조금만 넣어주세요,샷 더 넣어 드릴까요
3335,원래하던데로 해주세요,연하다 싶으면 더 넣어 드릴게요


In [ ]:
# 데이터 길이 자체가 짧기 때문에 그냥 split을 활용해도 충분할거 같은 느낌..?

encoder_input = []
for line in result_data["user"]:
    token = []
    words = line.split()
    for word in words:
        token.append(word)
    encoder_input.append(token)

# print(len(encoder_input))

decoder_input = []
for line in result_data["service"]:
    token = []
    words = line.split()
    for word in words:
        token.append(word)
    token.insert(0, "<start>")
    decoder_input.append(token)
# print(len(decoder_input))

decoder_output = []
for line in result_data["service"]:
    token=[]
    words = line.split()
    for word in words:
        token.append(word)
    token.append("<end>")
    decoder_output.append(token)

# print(len(decoder_output))

In [ ]:
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

[['아이스아메리카노', '하나요'], ['네', '텀블러에', '넣어주세요'], ['그란데', '사이즈로', '주세요']]
[['<start>', '테이크아웃하실', '건가요'], ['<start>', '텀블러', '할인', '300원', '해드릴게요'], ['<start>', '적립카드', '있으세요']]
[['테이크아웃하실', '건가요', '<end>'], ['텀블러', '할인', '300원', '해드릴게요', '<end>'], ['적립카드', '있으세요', '<end>']]


In [ ]:
tokenizer_text = Tokenizer()
tokenizer_text.fit_on_texts(encoder_input)
encoder_input = tokenizer_text.texts_to_sequences(encoder_input)

tokenizer_title = Tokenizer()
tokenizer_title.fit_on_texts(decoder_input)
tokenizer_title.fit_on_texts(decoder_output)
decoder_input = tokenizer_title.texts_to_sequences(decoder_input)
decoder_output = tokenizer_title.texts_to_sequences(decoder_output)

print(len(encoder_input))
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

3337
[[751, 55], [2, 97, 39], [253, 37, 1]]
[[1, 874, 60], [1, 169, 62, 300, 185], [1, 482, 76]]
[[874, 60, 2], [169, 62, 300, 185, 2], [482, 76, 2]]


In [ ]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_output = pad_sequences(decoder_output, padding="post")

In [ ]:
title_to_index = tokenizer_title.word_index
index_to_title = tokenizer_title.index_word

In [ ]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_output.shape)

print(len(encoder_input))
print(len(decoder_input))
print(len(decoder_output))

(3337, 14)
(3337, 19)
(3337, 19)
3337
3337
3337


In [ ]:
test_size = 800
encoder_input_train = encoder_input[ :-test_size]
decoder_input_train = decoder_input[ :-test_size]
decoder_output_train = decoder_output[ :-test_size]

encoder_input_test = encoder_input[test_size: ]
decoder_input_test = decoder_input[test_size: ]
decoder_output_test = decoder_output[test_size: ]

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Concatenate
from tensorflow.keras.models import Model

In [ ]:
encoder_inputs = Input(shape=(14,))
encoder_embed = Embedding(len(tokenizer_text.word_index)+1, 120)(encoder_inputs)
encoder_mask = Masking(mask_value=0)(encoder_embed)
encoder_outputs, h_state, c_state = LSTM(120, return_sequences=True, return_state=True)(encoder_mask)

In [ ]:
decoder_inputs = Input(shape=(20,))
decoder_embed = Embedding(len(tokenizer_title.word_index)+1, 120)(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)
decoder_lstm = LSTM(120, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])

In [ ]:
from attention import AttentionLayer
attn_layer = AttentionLayer()
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

decoder_concat_input = Concatenate()([decoder_outputs, attn_out])

decoder_dense = Dense(len(tokenizer_title.word_index)+1, activation= 'softmax')
decoder_softmax_outputs = decoder_dense(decoder_concat_input)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(x=[encoder_input_train, decoder_input_train],
          y=decoder_output_train,
          validation_data=([encoder_input_test, decoder_input_test], decoder_output_test),
          batch_size=128, epochs=50)

Epoch 1/50
20/20 [==============================] - 24s 1s/step - loss: 3.6340 - acc: 0.7051 - val_loss: 1.9054 - val_acc: 0.7272
Epoch 2/50
20/20 [==============================] - 21s 1s/step - loss: 1.7189 - acc: 0.7594 - val_loss: 1.8305 - val_acc: 0.7525
Epoch 3/50
20/20 [==============================] - 22s 1s/step - loss: 1.6233 - acc: 0.7739 - val_loss: 1.7269 - val_acc: 0.7742
Epoch 4/50
20/20 [==============================] - 21s 1s/step - loss: 1.5199 - acc: 0.7918 - val_loss: 1.6692 - val_acc: 0.7934
Epoch 5/50
20/20 [==============================] - 21s 1s/step - loss: 1.4548 - acc: 0.8074 - val_loss: 1.6423 - val_acc: 0.7931
Epoch 6/50
20/20 [==============================] - 21s 1s/step - loss: 1.4067 - acc: 0.8114 - val_loss: 1.5984 - val_acc: 0.7960
Epoch 7/50
20/20 [==============================] - 22s 1s/step - loss: 1.3665 - acc: 0.8123 - val_loss: 1.5770 - val_acc: 0.7964
Epoch 8/50
20/20 [==============================] - 22s 1s/step - loss: 1.3347 - acc: 0.81

In [ ]:
encoder_model = Model(encoder_inputs, [encoder_outputs, h_state, c_state])

encoder_h_state = Input(shape=(120,))
encoder_c_state = Input(shape=(120,))

pd_decoder_outputs,pd_h_state,pd_c_state = decoder_lstm(decoder_mask,
                                                        initial_state=[encoder_h_state,encoder_c_state])

pd_encoder_outputs = Input(shape=(14,120))
pd_attn_out, pd_attn_states = attn_layer([pd_encoder_outputs, pd_decoder_outputs])
pd_decoder_concat = Concatenate()([pd_decoder_outputs, pd_attn_out])

pd_decoder_softmax_outputs = decoder_dense(pd_decoder_concat)

decoder_model = Model([decoder_inputs, pd_encoder_outputs, encoder_h_state, encoder_c_state],
                      [pd_decoder_softmax_outputs, pd_h_state, pd_c_state])

In [ ]:
input_stc = input()
token_stc = input_stc.split()
encode_stc = tokenizer_text.texts_to_sequences([token_stc])
pad_stc = pad_sequences(encode_stc, maxlen=19, padding='post')

en_out, en_hidden, en_cell = encoder_model.predict(pad_stc)

predicted_seq = np.zeros((1,1))
predicted_seq[0,0] = title_to_index["<start>"]

decoded_stc = []

while True:
    output_words, h, c = decoder_model.predict([predicted_seq, en_out, en_hidden, en_cell])
    predicted_word = index_to_title[np.argmax(output_words[0,0])]
    
    if predicted_word == "<end>":
        break
    
    decoded_stc.append(predicted_word)

    predicted_seq = np.zeros((1,1))
    predicted_seq[0, 0] = np.argmax(output_words[0,0])

    en_hidden = h
    en_cell = c

print(" ".join(decoded_stc))

이천햅쌀라떼가 뭐죠?
네 비밀번호는 브랜드 방금 떨어져서 결제
